In [5]:
from huggingface_hub import notebook_login

notebook_login()


In [1]:
from modeling_esmc_plusplus import ESMC_plus_plus

In [6]:
model = ESMC_plus_plus.from_pretrained_esm('esmc_600m')
model

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

ESMC_plus_plus(
  (embed): Embedding(64, 1152)
  (transformer): TransformerStack(
    (blocks): ModuleList(
      (0-35): 36 x UnifiedTransformerBlock(
        (attn): MultiHeadAttention(
          (layernorm_qkv): Sequential(
            (0): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=1152, out_features=3456, bias=False)
          )
          (out_proj): Linear(in_features=1152, out_features=1152, bias=False)
          (q_ln): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
          (k_ln): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
          (rotary): RotaryEmbedding()
        )
        (ffn): Sequential(
          (0): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=1152, out_features=6144, bias=False)
          (2): SwiGLU()
          (3): Linear(in_features=3072, out_features=1152, bias=False)
        )
      )
    )
    (norm): LayerNorm((1152,), eps=1e-05, elementwise_aff

In [7]:
model.push_to_hub('Synthyra/ESMC_plus_plus_large', private=True)

model.safetensors:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Synthyra/ESMC_plus_plus_large/commit/cfee1dcaa4c95de1c9f08fef3430f7475cd60033', commit_message='Upload ESMC_plus_plus', commit_description='', oid='cfee1dcaa4c95de1c9f08fef3430f7475cd60033', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
from esm.tokenization import get_model_tokenizers
tokenizers = get_model_tokenizers()
tokenizers


c:\Users\Logan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

TokenizerCollection(sequence=EsmSequenceTokenizer(name_or_path='', vocab_size=33, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<cls>', 'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>', 'additional_special_tokens': ['|']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<cls>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<eos>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	31: AddedToken("|", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32: AddedToken("<mask>", rstrip=False, lst

In [11]:
from tokenizer import EsmSequenceTokenizer
tokenizer = EsmSequenceTokenizer()
tokenizer.vocab_size



c:\Users\Logan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


33

In [13]:
tokenizer.push_to_hub('Synthyra/ESMC_plus_plus_small', private=True)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\Logan\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Logan\.cache\huggingface\hub\models--Synthyra--ESMC_plus_plus_small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/Synthyra/ESMC_plus_plus_small/commit/1c1d2afaf9906a074bad0d60c6978a312ec9132c', commit_message='Upload tokenizer', commit_description='', oid='1c1d2afaf9906a074bad0d60c6978a312ec9132c', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
from transformers import AutoTokenizer

In [16]:
tokenizer = AutoTokenizer.from_pretrained('Synthyra/ESMC_plus_plus_small', trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.88k [00:00<?, ?B/s]

TypeError: expected str, bytes or os.PathLike object, not NoneType